In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

<font color="red" size="5">ch011 데이터프레임과 시리즈(pandas)</font>
# 9절. 데이터 그룹화 및 집계

In [3]:
from statsmodels.api import datasets
iris = datasets.get_rdataset('iris').data
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


## 9.1 Groupby
cf. pivot_table

In [16]:
# 1) 그룹화 방법1
#    df.groupby(by=[그룹핑할 열]).기초통계함수[함수적용해서 출력할 열들]
#                          [함수적용해서 출력할 열들] 생략시 그룹핑할열을 제외한 모든 열 적용
iris_grouped = iris.groupby(iris.Species)
iris_grouped = iris.groupby(['Species'])
iris_grouped = iris.groupby('Species')
for group, data in iris_grouped:
    print(group)
    display(data.sample(5).sort_index())

setosa


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
14,5.8,4.0,1.2,0.2,setosa
36,5.5,3.5,1.3,0.2,setosa
37,4.9,3.6,1.4,0.1,setosa
44,5.1,3.8,1.9,0.4,setosa
49,5.0,3.3,1.4,0.2,setosa


versicolor


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
64,5.6,2.9,3.6,1.3,versicolor
71,6.1,2.8,4.0,1.3,versicolor
79,5.7,2.6,3.5,1.0,versicolor
80,5.5,2.4,3.8,1.1,versicolor
99,5.7,2.8,4.1,1.3,versicolor


virginica


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
105,7.6,3.0,6.6,2.1,virginica
107,7.3,2.9,6.3,1.8,virginica
110,6.5,3.2,5.1,2.0,virginica
118,7.7,2.6,6.9,2.3,virginica
147,6.5,3.0,5.2,2.0,virginica


In [8]:
iris.groupby(by='Species').mean()[['Petal.Length', 'Petal.Width']]
iris.groupby(by='Species')[['Petal.Length', 'Petal.Width']].mean()

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [24]:
iris_grouped_sum = iris.groupby('Species').median()
#iris_grouped_sum.reset_index()
iris_grouped_sum

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3
virginica,6.5,3.0,5.55,2.0


In [25]:
iris_grouped_sum.loc[['setosa', 'versicolor']]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3


In [27]:
iris_grouped_sum2 = iris.groupby('Species', as_index=False).sum()
iris_grouped_sum2

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [28]:
iris_grouped_sum2[(iris_grouped_sum2.Species == 'setosa')|
                 (iris_grouped_sum2.Species == 'versicolor')]

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3


In [29]:
#    df.groupby(by=[그룹핑할 열]).[함수적용해서 출력할 열들].함수()
# 2) 그룹화 방법2
#    df.pivot_table(index=[그룹핑할 열],
#                   values=[함수적용해서 출력할 열들]
#                   aggfunc=함수(기본값:mean))

In [33]:
iris.groupby('Species')['Petal.Length', 'Petal.Width'].mean()
iris.pivot_table(index='Species',
                values=['Petal.Length', 'Petal.Width'],
                aggfunc='mean')

C:\Users\Admin\AppData\Local\Temp\ipykernel_8740\3702361755.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  iris.groupby('Species')['Petal.Length', 'Petal.Width'].mean()


,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [34]:
iris.groupby('Species').count() #결측치를 제외한 값의 갯수

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


In [37]:
iris.pivot_table(index='Species', 
                 #values=['Sepal.Length','Sepal.Width','Petal.Length','Petal.Width']
                aggfunc='count')

,Petal.Length,Petal.Width,Sepal.Length,Sepal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


In [38]:
iris.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


## 다중열로 그룹화

In [40]:
#iris로 다중열로 그룹화하기 위해 데이터 컬럼 추가(num필드)
iris.shape

(150, 5)

In [43]:
import numpy as np
np.ravel([[i]*25 for i in range(6)])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [72]:
iris['num'] = np.ravel([[i]*25 for i in range(6)])
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,NaN,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,1
50,7.0,3.2,4.7,1.4,versicolor,2
75,6.6,3.0,4.4,1.4,versicolor,3
100,6.3,3.3,6.0,2.5,virginica,4
125,7.2,3.2,6.0,1.8,virginica,5


In [47]:
# 각열의 임의의 데이터에 결측치를 할당
iris.iloc[0,0] = np.nan

In [57]:
import random
print(random.sample(range(149),3))
for col in range(iris.shape[1]-2): #iris.shape[1]-2:4
    iris.iloc[random.sample(range(150),col+1),col] = np.nan

[129, 105, 133]


In [58]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  144 non-null    float64
 1   Sepal.Width   142 non-null    float64
 2   Petal.Length  139 non-null    float64
 3   Petal.Width   136 non-null    float64
 4   Species       150 non-null    object 
 5   num           150 non-null    int32  
dtypes: float64(4), int32(1), object(1)
memory usage: 6.6+ KB


In [60]:
iris.groupby(by=['Species','num'])[['Petal.Length','Petal.Width']].count()

Petal.Length  Petal.Width
Species    num                           
setosa     0              23           22
           1              22           24
versicolor 2              23           22
           3              25           22
virginica  4              22           24
           5              24           22

In [63]:
iris.pivot_table(index = ['Species','num'],
                values = ['Petal.Length','Petal.Width'],
                aggfunc='count')

Petal.Length  Petal.Width
Species    num                           
setosa     0              23           22
           1              22           24
versicolor 2              23           22
           3              25           22
virginica  4              22           24
           5              24           22

In [66]:
iris.groupby('Species').describe().T

Species                setosa  versicolor  virginica
Sepal.Length count  48.000000   47.000000  49.000000
             mean    5.002083    5.963830   6.577551
             std     0.359367    0.511360   0.638117
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.200000
             50%     5.000000    6.000000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
Sepal.Width  count  47.000000   48.000000  47.000000
             mean    3.421277    2.787500   2.963830
             std     0.385011    0.307789   0.307462
             min     2.300000    2.000000   2.200000
             25%     3.150000    2.600000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.650000    3.000000   3.150000
             max     4.400000    3.400000   3.800000
Petal.Length count  45.000000   48.000000  46.000000
             mean    1.466667    4.262500   5.521739
             std     0.173205    0.457386   0.546062
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.350000   5.500000
             75%     1.600000    4.600000   5.800000
             max     1.900000    5.100000   6.900000
Petal.Width  count  46.000000   44.000000  46.000000
             mean    0.250000    1.311364   2.043478
             std     0.109036    0.185793   0.269693
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.425000   2.300000
             max     0.600000    1.700000   2.500000
num          count  50.000000   50.000000  50.000000
             mean    0.500000    2.500000   4.500000
             std     0.505076    0.505076   0.505076
             min     0.000000    2.000000   4.000000
             25%     0.000000    2.000000   4.000000
             50%     0.500000    2.500000   4.500000
             75%     1.000000    3.000000   5.000000
             max     1.000000    3.000000   5.000000

## 9.2 레이블 인코딩

In [73]:
iris.drop('num', axis=1, inplace=True)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,NaN,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,NaN,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,NaN,NaN,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [74]:
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,NaN,3.5,1.4,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


In [79]:
#레이블 인코딩을 지원하는 클래스 import
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() #객체 변수
iris['target'] = le.fit_transform(iris.Species)
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,target
0,NaN,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,0
50,7.0,3.2,4.7,1.4,versicolor,1
75,6.6,3.0,4.4,1.4,versicolor,1
100,6.3,3.3,6.0,2.5,virginica,2
125,7.2,3.2,6.0,1.8,virginica,2


# 10절. 데이터 구조 변경
- 와이드 포맷<->롱포맷
- 판다스 공식 문서의 melt : 
https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-by-melt
- Tidy Data(7page) : https://vita.had.co.nz/papers/tidy-data.pdf

In [99]:
from statsmodels.api import datasets
airquality = datasets.get_rdataset('airquality',
                                  package='datasets').data
airquality.head(2)

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2


## 10.1 melt를 이용한 언피벗팅(와이드포맷->롱포맷)

In [101]:
air_melted = airquality.melt(id_vars=['Month','Day'],
               var_name='변수명',
               value_name='값')
air_melted.iloc[::20]
air_melted.sort_values(by=['Month','Day'])

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0
...,...,...,...,...
610,9,29,Temp,76.0
152,9,30,Ozone,20.0
305,9,30,Solar.R,223.0
458,9,30,Wind,11.5


In [91]:
import pandas as pd
df = pd.read_csv(r'C:\ai_x\download\shareData\부동산\전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv', 
                encoding='cp949')
df

,지역,2013년12월,2014년1월,2014년2월,2014년3월,2014년4월,2014년5월,2014년6월,2014년7월,2014년8월,...,2014년11월,2014년12월,2015년1월,2015년2월,2015년3월,2015년4월,2015년5월,2015년6월,2015년7월,2015년8월
0,서울,18189,17925,17925,18016,18098,19446,18867,18742,19274,...,20242,20269,20670,20670,19415,18842,18367,18374,18152,18443
1,부산,8111,8111,9078,8965,9402,9501,9453,9457,9411,...,9208,9208,9204,9235,9279,9327,9345,9515,9559,9581
2,대구,8080,8080,8077,8101,8267,8274,8360,8360,8370,...,8439,8253,8327,8416,8441,8446,8568,8542,8542,8795
3,인천,10204,10204,10408,10408,10000,9844,10058,9974,9973,...,10020,10020,10017,9876,9876,9938,10551,10443,10443,10449
4,광주,6098,7326,7611,7346,7346,7523,7659,7612,7622,...,7752,7748,7752,7756,7861,7914,7877,7881,8089,8231
5,대전,8321,8321,8321,8341,8341,8341,8333,8333,8333,...,8067,8067,8067,8067,8067,8145,8272,8079,8079,8079
6,울산,8090,8090,8090,8153,8153,8153,8153,8153,8493,...,8891,8891,8526,8526,8629,9380,9192,9190,9190,9215
7,경기,10855,10855,10791,10784,10876,10646,10266,10124,10134,...,10356,10379,10391,10355,10469,10684,10685,10573,10518,10573
8,세종,7601,7600,7532,7814,7908,7934,8067,8067,8141,...,8592,8560,8560,8560,8555,8546,8546,8671,8669,8695
9,강원,6230,6230,6230,6141,6373,6350,6350,6268,6268,...,6365,6365,6348,6350,6182,6924,6846,6986,7019,7008


In [189]:
new_df = df.melt(id_vars='지역',
       var_name='연도월',
       value_name='평당분양가')
new_df.sample()

,지역,연도월,평당분양가
23,울산,2014년1월,8090


## 10.2 pivot_table을 이용한 피벗팅(롱포맷->와이드포맷)

In [95]:
air_melted.head()

,Month,Day,variable,value
0,5,1,Ozone,41.0
1,5,2,Ozone,36.0
2,5,3,Ozone,12.0
3,5,4,Ozone,18.0
4,5,5,Ozone,NaN


In [102]:
airquality2 = air_melted.pivot_table(index=['Month', 'Day'],
                      columns='변수명',
                      values='값')
airquality2.reset_index(inplace=True)
airquality2.columns.name = None

In [103]:
airquality2.head()

,Month,Day,Ozone,Solar.R,Temp,Wind
0,5,1,41.0,190.0,67.0,7.4
1,5,2,36.0,118.0,72.0,8.0
2,5,3,12.0,149.0,74.0,12.6
3,5,4,18.0,313.0,62.0,11.5
4,5,5,NaN,NaN,56.0,14.3


# 11절. 데이터프레임(시리즈)에 함수적용시키기
- apply(func, axis) : 데이터프레임이나 시리즈에서 함수 적용
- map(func) : 각 요소별 함수 적용(1차원 리스트, 시리즈에서만 가능)

In [104]:
new_df.head()

,지역,연도월,평당분양가
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098


In [5]:
import pandas as pd
member = pd.read_csv('data/ch11_member.csv',
                    comment='#')
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [109]:
def get_year(datestr):
    return int(datestr.split('-')[0])

In [110]:
get_year = lambda datestr : int(datestr.split('-')[0])

In [111]:
get_year('1999-05-01')

1999

In [112]:
get_month = lambda datestr : int(datestr.split('-')[1])

In [113]:
get_month('1999-05-01')

5

In [116]:
member['year'] = member['Birth'].apply(lambda datestr : int(datestr.split('-')[0]))
member.head()

,Name,Age,Email,Address,Birth,year
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999


In [134]:
X = iris.iloc[:,:-2]
X

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,NaN,3.5,1.4,0.2
1,4.9,3.0,1.4,NaN
2,4.7,3.2,1.3,0.2
3,4.6,3.1,NaN,NaN
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [135]:
#X의 열별 평균
avg = X.mean(axis=0)
avg

Sepal.Length    5.852083
Sepal.Width     3.055634
Petal.Length    3.774101
Petal.Width     1.200000
dtype: float64

In [136]:
X.iloc[0]-avg

Sepal.Length         NaN
Sepal.Width     0.444366
Petal.Length   -2.374101
Petal.Width    -1.000000
dtype: float64

In [137]:
X.apply(lambda x: round(x-avg, 2), axis=1)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,NaN,0.44,-2.37,-1.0
1,-0.95,-0.06,-2.37,NaN
2,-1.15,0.14,-2.47,-1.0
3,-1.25,0.04,NaN,NaN
4,-0.85,0.54,-2.37,-1.0
...,...,...,...,...
145,0.85,-0.06,1.43,1.1
146,0.45,-0.56,1.23,0.7
147,0.65,-0.06,1.43,0.8
148,0.35,0.34,1.63,1.1


## 11.2 applymap

In [138]:
X.applymap(np.round)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,NaN,4.0,1.0,0.0
1,5.0,3.0,1.0,NaN
2,5.0,3.0,1.0,0.0
3,5.0,3.0,NaN,NaN
4,5.0,4.0,1.0,0.0
...,...,...,...,...
145,7.0,3.0,5.0,2.0
146,6.0,2.0,5.0,2.0
147,6.0,3.0,5.0,2.0
148,6.0,3.0,5.0,2.0


## 11.3 map

In [141]:
member['month'] = member['Birth'].map(lambda datestr : int(datestr.split('-')[1]))
member

,Name,Age,Email,Address,Birth,year,month
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999,1
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999,12
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999,11
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999,5
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999,1


In [182]:
#연습하기(apply, map)
#new_df의 연도월 데이터를 연도와 월을 int로 분리하여 컬럼을 추가하세요.

In [181]:
new_df['연도'] = new_df['연도월'].apply(lambda datestr : int(datestr.split('년')[0]))
new_df['월'] = new_df['연도월'].apply(lambda datestr : datestr.split('년')[1][:-1])
new_df

,지역,연도월,평당분양가,연도,월
0,서울,2013년12월,18189,2013,12
1,부산,2013년12월,8111,2013,12
2,대구,2013년12월,8080,2013,12
3,인천,2013년12월,10204,2013,12
4,광주,2013년12월,6098,2013,12
...,...,...,...,...,...
352,전북,2015년8월,6580,2015,8
353,전남,2015년8월,6289,2015,8
354,경북,2015년8월,7037,2015,8
355,경남,2015년8월,7665,2015,8


In [201]:
def year_month_append(df):
    '''
    df를 받아 연도와 월 추가후 return
    '''
    df = df.copy()
    df['연도'] = int(df['연도월'].split('년')[0])
    df['월'] = int(df['연도월'].split('년')[1][:-1])
    return df

In [202]:
year_month_append(new_df.iloc[0])

지역             서울
연도월      2013년12월
평당분양가       18189
연도           2013
월              12
Name: 0, dtype: object

In [205]:
new_df = new_df.apply(year_month_append, axis=1)
new_df

,지역,연도월,평당분양가,연도,월
0,서울,2013년12월,18189,2013,12
1,부산,2013년12월,8111,2013,12
2,대구,2013년12월,8080,2013,12
3,인천,2013년12월,10204,2013,12
4,광주,2013년12월,6098,2013,12
...,...,...,...,...,...
352,전북,2015년8월,6580,2015,8
353,전남,2015년8월,6289,2015,8
354,경북,2015년8월,7037,2015,8
355,경남,2015년8월,7665,2015,8


# 12절. 일괄변경(결측치, 특정값)
## 12.1 fillna
- 대부분 결측치는 drop하거나, 평균값대체, 중위수대체, 그룹별 평균값, 그룹별 중위수, 이전값, 이후값 등으로 대체

In [7]:
import pandas as pd
import numpy as np
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                  [3,       4, np.nan, 1],
                  [np.nan, 3, np.nan, 5],
                  [np.nan, 3, np.nan, 4]],
                 columns=[list('ABCD')])
df

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [8]:
#모든 결측치를 0대체
df.fillna(0)

,A,B,C,D
0,0.0,2,0.0,0
1,3.0,4,0.0,1
2,0.0,3,0.0,5
3,0.0,3,0.0,4


In [9]:
#결측치를 이전행의 값으로 대체
df.fillna(method='ffill')

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [10]:
# 결측치를 열마다 특정값으로 대체
values = {'A':85, 'B':80, 'C':75, 'D':90}
df.fillna(value=values)

,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,75.0,5
3,85.0,3,75.0,4


In [11]:
#결측치를 열별 평균값으로 대체
df.mean(axis=0)
df.fillna(value=df.mean(axis=0))

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [12]:
# 결측치 대체를 2번째 결측치까지만 대체
print(values)
df.fillna(value=values, limit=2)

{'A': 85, 'B': 80, 'C': 75, 'D': 90}


,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,NaN,5
3,NaN,3,NaN,4


In [ ]:
#(1단계) iris 각 열마다 결측치를 20개씩 인위적으로 할당한다
#       결측치가 각 열마다 20개씩 확인
#(2-1단계) 결측치는 각 열의 평균으로 대체한다 : fillna사용
#(2-2단계) 결측치는 종별 열별 평균(groupby, pivot_table)으로 대체한다 : fillna사용

In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
iris = sns.load_dataset('iris')
rowlen, collen = iris.shape
rowlen, collen

(150, 5)

In [33]:
for col in range(collen-1):
    iris.iloc[random.sample(range(rowlen),20),col] = np.nan

In [35]:
#결측치 확인 - iris.isna(), iris.isnull():결측치인지 여부
#iris.isna().sum()
iris.isnull().sum()

sepal_length    20
sepal_width     20
petal_length    20
petal_width     20
species          0
dtype: int64

In [37]:
iris.tail()

,sepal_length,sepal_width,petal_length,petal_width,species
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,NaN,virginica
147,6.5,3.0,NaN,2.0,virginica
148,6.2,3.4,5.4,NaN,virginica
149,5.9,NaN,NaN,1.8,virginica


In [46]:
#결측치가 하나라도 있는 행 출력
iris[iris['sepal_length'].isna()|iris['sepal_width'].isna()|
iris['petal_length'].isna()|iris['petal_width'].isna()]

,sepal_length,sepal_width,petal_length,petal_width,species
6,NaN,NaN,1.4,0.3,setosa
7,NaN,NaN,1.5,0.2,setosa
9,4.9,3.1,NaN,0.1,setosa
13,4.3,3.0,1.1,NaN,setosa
14,NaN,4.0,1.2,0.2,setosa
...,...,...,...,...,...
143,6.8,3.2,5.9,NaN,virginica
146,6.3,2.5,5.0,NaN,virginica
147,6.5,3.0,NaN,2.0,virginica
148,6.2,3.4,5.4,NaN,virginica


In [39]:
#(2-1단계) 결측치는 각 열의 평균으로 대체한다 : fillna사용
iris.mean(numeric_only=True)
iris.iloc[:,:-1].mean()
iris.fillna(iris.iloc[:,:-1].mean())

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.500000,1.400000,0.200000,setosa
1,4.9,3.000000,1.400000,0.200000,setosa
2,4.7,3.200000,1.300000,0.200000,setosa
3,4.6,3.100000,1.500000,0.200000,setosa
4,5.0,3.600000,1.400000,0.200000,setosa
...,...,...,...,...,...
145,6.7,3.000000,5.200000,2.300000,virginica
146,6.3,2.500000,5.000000,1.183846,virginica
147,6.5,3.000000,3.677692,2.000000,virginica
148,6.2,3.400000,5.400000,1.183846,virginica


In [54]:
#(2-2단계) 결측치는 종별 열별 평균(groupby, pivot_table)으로 대체한다 : fillna사용x
iris_mean = iris.groupby('species').mean()
iris_mean

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [55]:
#fillna사용x 확인
iris.fillna(iris_mean)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [65]:
iris_mean # iris_mean.loc['setosa','sepal_length']

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [78]:
#iris한행을 매개변수로 받아 결측치가 있는 경우 종별 열별 중위수로 대체하는 함수
def iris_fillna(irisrow):
    irisrow=irisrow.copy()
    for col, value in irisrow.items():
        if isinstance(value, (int, float, np.number)) and np.isnan(value):
            #print(irisrow[col],'이 결측치')
            irisrow[col] = iris_mean.loc[irisrow.species,col]
    return irisrow

In [79]:
iris_fillna(iris.iloc[2])

sepal_length       4.7
sepal_width        3.2
petal_length       1.3
petal_width        0.2
species         setosa
Name: 2, dtype: object

In [80]:
#apply적용
iris.apply(iris_fillna, axis=1)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


## 12.2 replace(to_calue, new_value, inplace=T/F)
- to_value를 new_value로
- tp_value에 정규표현식 이용

In [84]:
s = pd.Series([0,1,2,3,np.nan])
s.replace(np.nan,0)
s.replace([0,1],99) #0이나 1을 99로 변경
s.replace([0,1],[100,101]) #0은 10으로, 1은 101로 변경

0    100.0
1    101.0
2      2.0
3      3.0
4      NaN
dtype: float64

In [90]:
df = pd.DataFrame({'A':['bat', 'foo', 'bait'],
                   'B':['abc', 'bar', 'xyz']})
df.replace(r'ba.+','XX',regex=True)

,A,B
0,XX,abc
1,foo,XX
2,XX,xyz


## 12.3 mask(조건)
- where(조건) : 조건이 True인 요소만 출력하고, 그 외 요소는 결측치로 출력
- mast(조건) : 조건이 False인 요소만 출력하고, 그 외 요소는 결측치로 출력

In [87]:
s = pd.Series(range(10,20))
s

0    10
1    11
2    12
3    13
4    14
5    15
6    16
7    17
8    18
9    19
dtype: int64

In [88]:
s.where(s>15)

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6    16.0
7    17.0
8    18.0
9    19.0
dtype: float64

In [89]:
s.mask(s>15)

0    10.0
1    11.0
2    12.0
3    13.0
4    14.0
5    15.0
6     NaN
7     NaN
8     NaN
9     NaN
dtype: float64

## 12.4 dropna
- 결측치가 있는 데이터 누락(행, 열)

In [91]:
df = pd.DataFrame({'name':['홍', '김', '이', np.nan],
                   'age':[np.nan, 20, 30, np.nan],
                   'born':['서울', '부산', np.nan, np.nan]})
df

,name,age,born
0,홍,NaN,서울
1,김,20.0,부산
2,이,30.0,NaN
3,NaN,NaN,NaN


In [92]:
df.dropna(how='all')#모두 결측치인 행만 삭제(axis=0기본값)

,name,age,born
0,홍,NaN,서울
1,김,20.0,부산
2,이,30.0,NaN


In [93]:
df.dropna() #결측치가 하나라도 있는 행 삭제

,name,age,born
1,김,20.0,부산


In [94]:
df.dropna(axis=1) #결측치가 하나라도 있는 열 삭제

""
0
1
2
3


In [96]:
df.dropna(subset=['name','age'])

,name,age,born
1,김,20.0,부산
2,이,30.0,NaN


In [97]:
s = pd.Series([1000, 7000, 5000],
             index = ['mango', 'apple', 'banana'])

In [98]:
s['mango'],s.mango

(1000, 1000)

In [100]:
s['mango'] = 2800
s['orange'] = 4000
s

mango     2800
apple     7000
banana    5000
orange    4000
dtype: int64

In [101]:
s['mango':'banana'] #슬라이싱[from:to] from부터 to까지(to포함)

mango     2800
apple     7000
banana    5000
dtype: int64

In [102]:
s[['mango','banana']]

mango     2800
banana    5000
dtype: int64

In [103]:
s.drop('banana')

mango     2800
apple     7000
orange    4000
dtype: int64

In [105]:
s.sort_index()

apple     7000
banana    5000
mango     2800
orange    4000
dtype: int64

In [106]:
s.sort_values() #오름차순정렬

mango     2800
orange    4000
banana    5000
apple     7000
dtype: int64

In [107]:
s.sort_values(ascending=False) #내림차순정렬

apple     7000
banana    5000
orange    4000
mango     2800
dtype: int64

In [112]:
s.to_frame().T

,mango,apple,banana,orange
0,2800,7000,5000,4000


# 13절. 연습문제-실습형

In [179]:
import seaborn as sns
import numpy as np
iris = sns.load_dataset('iris')
type(iris)

pandas.core.frame.DataFrame

In [180]:
#1.iris 데이터에서 처음 다섯개 행만 출력하세요
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [181]:
#2.iris 데이터를 데이터프레임 변수인 독립변수 X와 종속변수y로 나누세요 . 종속변수는species열입니다
#iloc이용
X = iris.iloc[:,:-1]
y = iris.iloc[:,-1:]

In [183]:
#3.iris 데이터에서 처음 50 개행을 빼내서 temp 변수에 저장하세요
temp = iris.iloc[:50, :]
temp

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5.0,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
9,4.9,3.1,1.5,0.1,setosa


In [184]:
#4. 3 번에서 선택한 데이터프레임의 요약정보를 출력하세요 . 모든 열에 대해 요약정보가 출력되어야 합니다
temp.describe(include='all')

,sepal_length,sepal_width,petal_length,petal_width,species
count,50.00000,50.000000,50.000000,50.000000,50
unique,NaN,NaN,NaN,NaN,1
top,NaN,NaN,NaN,NaN,setosa
freq,NaN,NaN,NaN,NaN,50
mean,5.00600,3.428000,1.462000,0.246000,NaN
std,0.35249,0.379064,0.173664,0.105386,NaN
min,4.30000,2.300000,1.000000,0.100000,NaN
25%,4.80000,3.200000,1.400000,0.200000,NaN
50%,5.00000,3.400000,1.500000,0.200000,NaN
75%,5.20000,3.675000,1.575000,0.300000,NaN


In [185]:
#5.versicolor 종의 데이터만 iris_versicolor 변수에 저장하세요
iris_grouped = iris.groupby('species')
iris_versicolor = iris_grouped.get_group('versicolor')
iris_versicolor

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor
53,5.5,2.3,4.0,1.3,versicolor
54,6.5,2.8,4.6,1.5,versicolor
55,5.7,2.8,4.5,1.3,versicolor
56,6.3,3.3,4.7,1.6,versicolor
57,4.9,2.4,3.3,1.0,versicolor
58,6.6,2.9,4.6,1.3,versicolor
59,5.2,2.7,3.9,1.4,versicolor


In [155]:
#6. 2 번의 X 와 Y 변수를 합해서 iris_df 데이터 프레임으로 만드세요(y가 시리즈여도 concat가능)
iris_df = pd.DataFrame(data=X,
                      columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
iris_df['species'] = Y
iris_df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [186]:
#7. iris데이터의 각 열 평균값을 출력하세요
iris.iloc[:,:-1].mean(axis=0)

sepal_length    5.843333
sepal_width     3.057333
petal_length    3.758000
petal_width     1.199333
dtype: float64

In [165]:
#8. iris데이터의 각 열들 사이의 상관계수를 출력하세요
iris_corr = iris_df.groupby(by='species').corr()[['sepal_length','sepal_width','petal_length','petal_width']]
iris_corr

sepal_length  sepal_width  petal_length  petal_width
species                                                                      
setosa     sepal_length      1.000000     0.742547      0.267176     0.278098
           sepal_width       0.742547     1.000000      0.177700     0.232752
           petal_length      0.267176     0.177700      1.000000     0.331630
           petal_width       0.278098     0.232752      0.331630     1.000000
versicolor sepal_length      1.000000     0.525911      0.754049     0.546461
           sepal_width       0.525911     1.000000      0.560522     0.663999
           petal_length      0.754049     0.560522      1.000000     0.786668
           petal_width       0.546461     0.663999      0.786668     1.000000
virginica  sepal_length      1.000000     0.457228      0.864225     0.281108
           sepal_width       0.457228     1.000000      0.401045     0.537728
           petal_length      0.864225     0.401045      1.000000     0.322108
           petal_width       0.281108     0.537728      0.322108     1.000000

In [188]:
#9. iris데이터에서 각 요소들과 변수별 평균과의 차이를 출력하세요(이미지 참고)
iris_mean = iris.iloc[:,:-1].mean()
iris.iloc[:,:-1].apply(lambda x : x-iris_mean, axis=1)

,sepal_length,sepal_width,petal_length,petal_width
0,-0.743333,0.442667,-2.358,-0.999333
1,-0.943333,-0.057333,-2.358,-0.999333
2,-1.143333,0.142667,-2.458,-0.999333
3,-1.243333,0.042667,-2.258,-0.999333
4,-0.843333,0.542667,-2.358,-0.999333
...,...,...,...,...
145,0.856667,-0.057333,1.442,1.100667
146,0.456667,-0.557333,1.242,0.700667
147,0.656667,-0.057333,1.442,0.800667
148,0.356667,0.342667,1.642,1.100667


In [ ]:
#10. iris데이터의 종별 평균을 출력하세요


In [ ]:
#11. 
